Hui-Kang Tong admin, [2 Apr 2018 at 19:31:29]:
ok we can now automatically generate midi files, convert midi files to wave files (with hk’s mac)

However to create training data, we need many data (audio samples) with the same target/label (note). So these are a few ways to vary it:
- note velocity
- note volume
- noise
- onset timing (one slightly later than the other)
- soundfont (which mean instrument)

So I will write a notebook which
- leave a space for the function to create .mid file with note
- use fluidsynth to convert this .mid file .wav
- call this .mid file, identify onset with our onset function, and splice 4096 entries from it
- leave a space for the training function
- delete the .mid file and .wav file
- repeat

What needs to be considered/written
what kind of model are we using, how do we train such a model, what sort of instructions does the .mid files have

In [1]:
from midiutil.MidiFile import MIDIFile
from itertools import combinations
import os
import subprocess

scales = range(4,5) #C1 to C4
no_of_notes = range(2, 5) #2 notes to 4 notes

channel = 0
time = 0
duration = 2
tempo = 100
volume = 100

In [2]:
from midiutil.MidiFile import MIDIFile
from itertools import combinations
import os
import subprocess

def generate_and_train(no_,track,com,channel=0):
# for each cycle

    # generate and save midifile
    mf = MIDIFile(no_)

    for num in range(no_):
        mf.addNote(track[num], channel, com[num], time, duration, volume)

    code = []
    for num in com:
        code.append(num)
    while len(code) < 4:
        code.append("x")

    filename = "C{}_{}_{}_{}_{}".format(scale, code[0], code[1], code[2], code[3])
    with open("midifile_sch/{}.mid".format(filename), "wb") as outf:
        mf.writeFile(outf)

    # generate sound
    subprocess.run("fluidsynth -F soundfile_sch/{}.wav ~/mgen/Sonatina_Symphonic_Orchestra.sf2 midifile_sch/{}.mid".format(filename,filename),shell=True)
    subprocess.run("rm midifile_sch/{}.mid".format(filename),shell=True) # remove midifile

    # CALL SOME TRAINING FUNCTION HERE, UPDATE THE MODEL
    subprocess.run("rm soundfile_sch/{}.wav".format(filename),shell=True) # remove soundfile

In [ ]:
# can loop in other ways or random
for scale in scales:
    for no_ in no_of_notes:
        
        pitch = range(12*(scale+1), 12*(scale+2))
        comb = [x for x in combinations(pitch, no_)]
        track = range(no_)
        
        count = 0    
        for com in comb:
            generate_and_train(no_,track,com,channel=channel)
            
            count += 1